In [1]:
import pandas as pd
from correction.compute import *
from correction.helpers import *

# Prepare your dataset
The core code use different functions for correct the light curve, in the following blocks we'll use theses functions for correct a data examples.

In [2]:
detections = pd.read_parquet("../data_examples/parquets/10_objects_detections.parquet")
display(detections.head())

non_detections = pd.read_parquet("../data_examples/parquets/10_objects_non_detections.parquet", 
                                 columns=["objectId", "fid", "jd", "diffmaglim"])
display(non_detections.head())

,objectId,jd,fid,pid,diffmaglim,pdiffimfilename,programpi,programid,candid,isdiffpos,...,simag3,szmag3,sgscore3,distpsnr3,nmtchps,rfid,jdstartref,jdendref,nframesref,parent_candid
0,ZTF18aaioozo,2.458273e+06,1,518415875315,20.182840,ztf_20180603415822_000683_zg_c14_o_q2_scimrefd...,Kulkarni,1,518415875315010002,f,...,15.847300,15.798900,0.998750,8.699934,15,683120153,2.458205e+06,2.458231e+06,15,0
1,ZTF18aaurrwb,2.458279e+06,1,524348253515,20.939032,ztf_20180609348229_000723_zg_c09_o_q4_scimrefd...,Kulkarni,1,524348253515015051,t,...,21.393101,21.497999,0.751875,19.763674,9,723120135,2.458204e+06,2.458207e+06,16,0
2,ZTF18abadmxd,2.458282e+06,1,527431606115,20.560465,ztf_20180612431609_000804_zg_c16_o_q2_scimrefd...,Kulkarni,1,527431606115010124,f,...,16.569500,16.329300,0.974434,8.267725,37,804120161,2.458229e+06,2.458277e+06,15,0
3,ZTF18abadvod,2.458282e+06,2,527448113315,20.925859,ztf_20180612448102_000687_zr_c09_o_q2_scimrefd...,Kulkarni,1,527448113315010023,f,...,21.053900,20.278700,0.088690,4.907485,53,687120233,2.458199e+06,2.458257e+06,15,0
4,ZTF18abadvtf,2.458282e+06,2,527450573615,20.881939,ztf_20180612450556_000730_zr_c10_o_q1_scimrefd...,Kulkarni,1,527450573615015012,t,...,20.822399,20.271200,0.625548,7.855210,34,730120236,2.458223e+06,2.458280e+06,15,0


,objectId,fid,jd,diffmaglim
369,ZTF18aazeood,2,2.458384e+06,20.273001
137,ZTF18aaioozo,1,2.458661e+06,20.883699
449,ZTF18abadmxd,2,2.458313e+06,20.606800
394,ZTF18abadmxd,2,2.458321e+06,20.769300
347,ZTF18aazeood,1,2.458434e+06,19.930201


First, we need a modified julian dates.

In [3]:
detections["mjd"] = detections.jd - 2400000.5
del detections["jd"]

non_detections["mjd"] = non_detections.jd - 2400000.5
del non_detections["jd"]

2020-06-24 11:16:31 INFO numexpr.utils._init_num_threads: NumExpr defaulting to 4 threads.


# Getting a LC corrected
Apply the `apply_correction_df` functon for all unique [objectId, fid] pairs.

In [4]:
corrected = detections.groupby(["objectId", "fid"]).apply(apply_correction_df)
corrected.reset_index(inplace=True)
display(corrected.head())

,objectId,fid,candid,pid,diffmaglim,pdiffimfilename,programpi,programid,isdiffpos,tblid,...,jdstartref,jdendref,nframesref,parent_candid,mjd,corrected,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ext,dubious
0,ZTF17aaaecgt,1,550436093915015009,550436093915,20.544632,ztf_20180705436100_000801_zg_c10_o_q4_scimrefd...,Kulkarni,1,1.0,9,...,2.458232e+06,2.458289e+06,15,0,58304.436099,True,16.510900,0.014875,0.018298,False
1,ZTF17aaaecgt,1,1209408473915010011,1209408473915,20.269983,ztf_20200424408472_000801_zg_c10_o_q4_scimrefd...,Kulkarni,1,-1.0,11,...,2.458232e+06,2.458289e+06,15,0,58963.408472,True,17.879532,0.055861,0.067333,False
2,ZTF17aaaecgt,1,1004203723915015017,1004203723915,20.481764,ztf_20191002203727_000801_zg_c10_o_q4_scimrefd...,TESS,1,1.0,17,...,2.458232e+06,2.458289e+06,15,0,58758.203727,True,16.742756,0.009440,0.016224,False
3,ZTF17aaaecgt,1,978289493915010013,978289493915,20.881487,ztf_20190906289468_000801_zg_c10_o_q4_scimrefd...,TESS,1,-1.0,13,...,2.458232e+06,2.458289e+06,15,0,58732.289491,True,17.739686,0.052527,0.062059,False
4,ZTF17aaaecgt,1,1009243913915015013,1009243913915,20.051744,ztf_20191007243912_000801_zg_c10_o_q4_scimrefd...,TESS_Prince,3,1.0,13,...,2.458232e+06,2.458289e+06,15,0,58763.243912,True,16.697498,0.016639,0.020905,False


# Getting magnitude statistics for LC corrected

In [5]:
magstats = corrected.groupby(["objectId", "fid"]).apply(apply_mag_stats)
magstats.reset_index(inplace=True)
display(magstats)

,objectId,fid,corrected,nearZTF,nearPS1,stellarZTF,stellarPS1,stellar,ndet,ndubious,...,magpsf_corr_first,magpsf_corr_last,magap_mean,magap_median,magap_max,magap_min,magap_first,magap_last,first_mjd,last_mjd
0,ZTF17aaaecgt,1,True,True,True,True,True,True,231,0,...,16.510900,17.232855,18.321594,18.237900,20.348900,17.119900,17.825899,18.269400,58304.436099,58296.463981
1,ZTF17aaaecgt,2,True,True,True,True,True,True,208,3,...,14.530271,15.516477,16.190769,16.024151,17.957899,14.922700,17.319500,15.286500,58769.145382,58350.374699
2,ZTF18aaioozo,1,True,True,True,True,True,True,98,0,...,16.011487,15.755228,17.895746,17.797699,19.234100,17.003799,18.106199,18.229099,58272.415880,58258.363368
3,ZTF18aaioozo,2,True,True,True,True,True,True,60,0,...,15.965566,15.880039,17.805830,17.736549,18.667200,17.453600,17.654200,18.328800,58647.308565,58299.373623
4,ZTF18aaurrwb,1,False,False,False,True,False,False,3,0,...,NaN,NaN,20.358467,20.370001,20.608601,20.096800,20.370001,20.608601,58278.348252,58258.374213
5,ZTF18aazeood,1,True,True,True,True,True,True,367,0,...,16.678734,16.504788,18.260881,18.181900,19.944500,17.461500,17.867201,18.579300,58781.187234,58317.381331
6,ZTF18aazeood,2,True,True,True,True,True,True,396,0,...,15.785927,15.942050,17.641266,17.561350,19.102301,16.823500,18.316999,17.561600,58316.444780,58750.215324
7,ZTF18abadmxd,1,True,True,True,True,True,True,3,0,...,14.717818,14.633466,18.612566,18.576799,19.036900,18.224001,18.576799,18.224001,58281.431609,58320.457442
8,ZTF18abadvod,1,True,True,True,True,True,True,1,0,...,18.088278,18.088278,20.612000,20.612000,20.612000,20.612000,20.612000,20.612000,58390.151840,58390.151840
9,ZTF18abadvod,2,True,True,True,True,True,True,7,0,...,17.372926,17.368911,19.964672,19.937099,20.225201,19.723200,20.225201,19.937099,58281.448113,58314.423484


# Getting objects statistics

In [6]:
objstats = apply_object_stats_df(corrected, magstats)
objstats.reset_index(inplace=True)
display(objstats)

,objectId,ndethist,ncovhist,mjdstarthist,mjdendhist,meanra,meandec,sigmara,sigmadec,firstmjd,...,nearZTF,nearPS1,stellar,corrected,ndet,ndubious,g-r_max,g-r_max_corr,g-r_mean,g-r_mean_corr
0,ZTF17aaaecgt,569.0,1085.0,58101.123472,59009.280961,315.464120,55.418098,0.000085,0.000066,58289.408912,...,True,True,True,True,439,3,2.268444,2.095278,2.126385,2.158688
1,ZTF18aaioozo,1032.0,1941.0,58218.421956,59009.276736,275.371236,36.742764,0.000067,0.000060,58244.431273,...,True,True,True,True,158,0,-0.458384,-0.047698,0.063620,0.035294
2,ZTF18aaurrwb,12.0,509.0,58227.365694,58278.362986,256.997487,40.866430,0.000310,0.000133,58258.374213,...,False,False,False,False,3,0,NaN,NaN,NaN,NaN
3,ZTF18aazeood,1048.0,1893.0,58078.177292,59010.477998,327.181484,49.716779,0.000078,0.000059,58278.410035,...,True,True,True,True,763,0,0.567499,0.648202,0.587791,0.695056
4,ZTF18abadmxd,5.0,153.0,58281.431609,58320.457442,339.196380,57.996095,0.000121,0.000053,58281.431609,...,True,True,True,True,3,0,NaN,NaN,NaN,NaN
5,ZTF18abadvod,10.0,479.0,58281.448113,58669.405718,309.249942,34.754976,0.000044,0.000074,58281.448113,...,True,True,True,True,8,0,0.972050,0.719368,0.577728,0.693038
6,ZTF18abadvtf,1.0,16.0,58281.450579,58281.450579,314.302906,41.961783,NaN,NaN,58281.450579,...,False,False,False,False,1,0,NaN,NaN,NaN,NaN
7,ZTF18abbklxk,1.0,4.0,58285.333715,58285.333715,274.962513,-6.724415,NaN,NaN,58285.333715,...,True,True,True,True,1,0,NaN,NaN,NaN,NaN
8,ZTF18abdkytr,5.0,331.0,58301.322708,58639.441701,295.403046,29.808414,0.000022,0.000035,58301.322708,...,False,True,False,False,3,1,0.518068,NaN,0.565115,NaN
9,ZTF18abgptvl,1.0,233.0,58435.261308,58435.261308,347.973290,58.613742,0.000478,0.000120,58312.465509,...,True,True,True,True,2,0,0.732382,0.690671,0.732382,0.690671


# Doing dm/dt

In [10]:
dmdt = do_dmdt_df(magstats, non_detections)
display(dmdt)

close_nondet  dmdt_first  dm_first  sigmadm_first   dt_first
objectId     fid                                                              
ZTF17aaaecgt 1           False   -0.740703 -2.366047     -20.159594   3.108993
             2           False   -0.501988 -3.076042     -20.294884   5.931667
ZTF18aaioozo 1           False   -0.154541 -2.681528     -20.852516  17.035857
             2           False   -0.956442 -2.926596     -20.447359   2.982465
ZTF18aaurrwb 1           False   -0.067345 -0.541382     -20.673570   6.090289
ZTF18aazeood 1           False   -0.677907 -2.842726     -20.461081   4.072396
             2           False   -2.085179 -2.439274     -20.541516   1.084074
ZTF18abadmxd 1           False   -2.276319 -2.303377     -20.627207   0.976701
ZTF18abadvod 1           False   -0.084975 -0.301922     -20.552155   1.000012
             2           False   -0.545535 -0.740147     -20.702034   1.084769
ZTF18abadvtf 2           False   -1.972541 -2.198999     -20.761900   1.086771
ZTF18abbklxk 2           False   -0.156795 -0.454407     -20.017875   1.961030
ZTF18abdkytr 1           False   -1.320684 -1.308931     -19.794093   0.843368
             2           False         NaN       NaN            NaN        NaN
ZTF18abgptvl 1           False   -0.452267 -1.123180     -20.624476   2.099769
             2           False   -0.728268 -0.968462     -19.711232   1.055648

In [8]:
magstats.join(dmdt, on=["objectId", "fid"])

,objectId,fid,corrected,nearZTF,nearPS1,stellarZTF,stellarPS1,stellar,ndet,ndubious,...,magap_min,magap_first,magap_last,first_mjd,last_mjd,close_nondet,dmdt_first,dm_first,sigmadm_first,dt_first
0,ZTF17aaaecgt,1,True,True,True,True,True,True,231,0,...,17.119900,17.825899,18.269400,58304.436099,58296.463981,False,-0.740703,-2.366047,-20.159594,3.108993
1,ZTF17aaaecgt,2,True,True,True,True,True,True,208,3,...,14.922700,17.319500,15.286500,58769.145382,58350.374699,False,-0.501988,-3.076042,-20.294884,5.931667
2,ZTF18aaioozo,1,True,True,True,True,True,True,98,0,...,17.003799,18.106199,18.229099,58272.415880,58258.363368,False,-0.154541,-2.681528,-20.852516,17.035857
3,ZTF18aaioozo,2,True,True,True,True,True,True,60,0,...,17.453600,17.654200,18.328800,58647.308565,58299.373623,False,-0.956442,-2.926596,-20.447359,2.982465
4,ZTF18aaurrwb,1,False,False,False,True,False,False,3,0,...,20.096800,20.370001,20.608601,58278.348252,58258.374213,False,-0.067345,-0.541382,-20.673570,6.090289
5,ZTF18aazeood,1,True,True,True,True,True,True,367,0,...,17.461500,17.867201,18.579300,58781.187234,58317.381331,False,-0.677907,-2.842726,-20.461081,4.072396
6,ZTF18aazeood,2,True,True,True,True,True,True,396,0,...,16.823500,18.316999,17.561600,58316.444780,58750.215324,False,-2.085179,-2.439274,-20.541516,1.084074
7,ZTF18abadmxd,1,True,True,True,True,True,True,3,0,...,18.224001,18.576799,18.224001,58281.431609,58320.457442,False,-2.276319,-2.303377,-20.627207,0.976701
8,ZTF18abadvod,1,True,True,True,True,True,True,1,0,...,20.612000,20.612000,20.612000,58390.151840,58390.151840,False,-0.084975,-0.301922,-20.552155,1.000012
9,ZTF18abadvod,2,True,True,True,True,True,True,7,0,...,19.723200,20.225201,19.937099,58281.448113,58314.423484,False,-0.545535,-0.740147,-20.702034,1.084769


# Getting other tables

In [9]:
display(get_data_quality(detections))
display(get_ss_ztf(detections))
display(get_ps1_ztf(detections))

,aimage,aimagerat,bimage,bimagerat,candid,chipsf,classtar,fid,fwhm,mindtoedge,nbad,nneg,objectId,scorr,seeratio,sky,sumrat,xpos,ypos
0,0.645,0.483742,0.607,0.455243,518415875315010002,15.384957,1.000,1,1.333354,1423.983154,0,5,ZTF18aaioozo,14.042130,2.000000,-0.143244,0.952911,1423.983154,1437.459717
1,1.276,0.238951,1.130,0.211610,524348253515015051,2.088027,0.290,1,5.340000,1068.812256,0,2,ZTF18aaurrwb,7.410218,0.475763,0.343839,0.995277,1068.812256,2001.442993
2,0.617,0.440604,0.427,0.304923,527431606115010124,15.689263,0.993,1,1.400352,583.100525,0,10,ZTF18abadmxd,7.323398,2.000000,-0.706248,0.797539,2489.399414,1976.290771
3,0.762,0.604016,0.598,0.474018,527448113315010023,2.979355,0.906,2,1.261556,656.595276,0,5,ZTF18abadvod,5.268152,2.000000,-0.841650,0.977595,656.595276,1463.585449
4,1.161,0.935257,1.051,0.846645,527450573615015012,5.061202,0.509,2,1.241370,630.814209,4,0,ZTF18abadvtf,5.147875,2.000000,2.840725,1.000000,2441.685791,1561.271118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,0.844,0.078076,0.813,0.075208,596374693915010002,1126.500000,0.989,2,10.810000,1086.810059,0,7,ZTF17aaaecgt,18.172100,0.216840,-1.484480,0.954546,1985.689941,1595.380005
267,0.791,0.250316,0.748,0.236709,557365133915015018,15.582900,0.984,1,3.160000,1105.560059,0,2,ZTF17aaaecgt,24.146900,0.800826,0.155358,1.000000,1966.939941,1657.959961
795,0.733,0.245151,0.642,0.214716,542463983915010002,8.456710,0.984,1,2.990000,1093.119995,0,5,ZTF17aaaecgt,11.570000,0.800017,-0.143995,0.945132,1979.380005,1800.949951
916,0.543,0.429994,0.461,0.365060,891442386215010057,11.692700,0.981,1,1.262810,348.653992,0,8,ZTF18aazeood,8.402860,2.000000,-0.806836,0.521772,1507.630005,2731.850098


,candid,objectId,ssdistnr,ssmagnr,ssnamenr
0,518415875315010002,ZTF18aaioozo,-999.0,-999.0,null
1,524348253515015051,ZTF18aaurrwb,-999.0,-999.0,null
2,527431606115010124,ZTF18abadmxd,-999.0,-999.0,null
3,527448113315010023,ZTF18abadvod,-999.0,-999.0,null
4,527450573615015012,ZTF18abadvtf,-999.0,-999.0,null
...,...,...,...,...,...
48,596374693915010002,ZTF17aaaecgt,NaN,NaN,None
267,557365133915015018,ZTF17aaaecgt,NaN,NaN,None
795,542463983915010002,ZTF17aaaecgt,NaN,NaN,None
916,891442386215010057,ZTF18aazeood,NaN,NaN,None


,objectId,fid,sgscore1,szmag1,srmag2,nmtchps,distpsnr3,objectidps2,szmag2,simag3,...,candid,distpsnr2,sgmag3,srmag3,simag1,objectidps1,simag2,objectidps3,szmag3,srmag1
0,ZTF17aaaecgt,1,0.968958,13.609900,-999.000000,15.0,4.200218,1.745032e+17,17.557699,14.767500,...,5.354089e+17,2.523251,16.523800,15.417800,13.5874,1.745032e+17,18.408701,1.745032e+17,14.430700,14.6274
1,ZTF17aaaecgt,2,0.968958,13.609900,-999.000000,15.0,4.168311,1.745032e+17,17.557699,14.767500,...,5.493630e+17,2.670777,16.523800,15.417800,13.5874,1.745032e+17,18.408701,1.745032e+17,14.430700,14.6274
2,ZTF18aaioozo,1,1.000000,15.926400,19.699100,15.0,8.699934,1.520928e+17,18.337299,15.847300,...,4.904313e+17,3.283490,16.526800,16.027800,15.7308,1.520928e+17,18.747499,1.520928e+17,15.798900,15.8468
3,ZTF18aaioozo,2,1.000000,15.926400,19.699100,15.0,8.699934,1.520928e+17,18.337299,15.847300,...,4.904621e+17,3.283490,16.526800,16.027800,15.7308,1.520928e+17,18.747499,1.520928e+17,15.798900,15.8468
4,ZTF18aaurrwb,1,0.500000,12.250000,-999.000000,9.0,19.763674,1.570426e+17,-999.000000,21.393101,...,5.043742e+17,18.036150,-999.000000,22.283501,12.4370,1.570426e+17,21.403400,1.570326e+17,21.497999,12.7550
5,ZTF18aazeood,1,0.977417,15.168900,20.883699,43.0,7.643254,1.676633e+17,19.596201,20.756500,...,5.244100e+17,4.881994,-999.000000,21.646799,15.3179,1.676633e+17,20.057600,1.676533e+17,20.228399,15.7270
6,ZTF18aazeood,2,0.977417,15.168900,20.883699,46.0,7.555435,1.676633e+17,19.596201,20.212299,...,5.354526e+17,4.467855,-999.000000,21.332800,15.3179,1.676633e+17,20.057600,1.676633e+17,19.627399,15.7270
7,ZTF18abadmxd,1,0.500000,12.406000,-999.000000,37.0,8.267725,1.775934e+17,19.564800,16.569500,...,5.274316e+17,7.964578,17.815800,16.985100,12.7650,1.775934e+17,19.941200,1.775934e+17,16.329300,13.3100
8,ZTF18abadvod,1,0.989060,16.629700,-999.000000,53.0,4.851420,1.497031e+17,18.368999,21.053900,...,6.361518e+17,4.042696,-999.000000,21.773600,16.8909,1.497031e+17,19.051201,1.497031e+17,20.278700,17.4188
9,ZTF18abadvod,2,0.989060,16.629700,-999.000000,53.0,4.907485,1.497031e+17,18.368999,21.053900,...,5.274481e+17,3.965276,-999.000000,21.773600,16.8909,1.497031e+17,19.051201,1.497031e+17,20.278700,17.4188
